### Yahoo Finance API

In [11]:
import requests
import re
import pandas as pd
from StringIO import StringIO
import time 

import os

os.chdir('C:\Working\USA\datafiles')

def write_txt(filename, filetext):
# Write label to txt file
    with open(filename, 'w') as text_file:
        text_file.write(filetext)

In [47]:
tickers = ['^DJI', '^GSPC', '^IXIC']
start = '1104537600'
base = 'https://finance.yahoo.com/quote/'
base2 = 'https://query1.finance.yahoo.com/v7/finance/download/'
end = int(time.time())
dates = '?period1={}&period2={}'.format(start, end)
param = '&interval=1d&filter=history&frequency=1d'
d = {}

for t in tickers:
    url = '{}{}/history{}{}'.format(base, t, dates, param)
    s = requests.Session()
    r = s.get(url)
    regex = '"CrumbStore":{"crumb":"(.+?)"},'
    pattern = re.compile(regex)
    crumb = re.findall(pattern, r.content)[0]
    param2 = '{}{}&interval=1d&events=history&crumb={}/Q'.format(t, dates, crumb)
    url2 = '{}{}'.format(base2, param2)
    data = s.post(url2)
    csv = StringIO(data.content)
    d[t[1:]] = pd.read_table(csv, sep=',', 
                             parse_dates=['Date']).set_index('Date')['Adj Close']

In [48]:
df = pd.DataFrame(d)
df = df[df['GSPC'].str.contains('null') == False].apply(pd.to_numeric)

In [49]:
c_names = ['djia', 'sp', 'nsdq']
df.columns = c_names
df.loc[:, 'djia'] = df['djia'].divide(1000)
for col in c_names:
    df.loc[:,col+'_ma'] = df[col].rolling(253).mean()
df = df.dropna()
for col in c_names:
    df.loc[:,col+'_ix'] = df[col].divide(df[col].iloc[0] / 100)
df.to_csv('stocks.csv')

# Generate label text
d = {}
for col in c_names:
    for t, v in {'day': -2, 'wk': -6}.iteritems():
        d[col+'_ch'+t] = round((df[col][-1] - df[col][v]) / df[col][v] * 100, 2)
        if d[col+'_ch'+t] > 0:
            d[col+'_ar'+t] = '\color{green!80!black} $ \\blacktriangle $ \\normalcolor'
        else:
            d[col+'_ar'+t] = '\color{red} $\\blacktriangledown$ \\normalcolor'
    d[col+'_mad'] = round((df[col][-1] - df[col+'_ma'][-1]) / df[col+'_ma'][-1] * 100, 2)
    if d[col+'_mad'] > 0:
        d[col+'_mat'] = 'above moving average'
    else:
        d[col+'_mat'] = 'below moving average'
df.loc[:, 'djia'] = df['djia'].multiply(1000)
d1 = df.index[-1].strftime('%b %#d, %Y')
for col in c_names:    
    d[col+'_lt'] = round(df[col][-1], 2)
    d[col+'_t'] = 'One-day change: {} {}\%; \\\ One-week change: {} {}\%'.format(
            d[col+'_arday'], d[col+'_chday'], d[col+'_arwk'], d[col+'_chwk'])
    d[col+'_t2'] = '{}: {:,}; {}\% {}'.format(d1, d[col+'_lt'], d[col+'_mad'], d[col+'_mat'])
    write_txt(col+'_ch.txt', d[col+'_t'])
    write_txt(col+'.txt', d[col+'_t2'])